In [ ]:
## prevent colab shutdown 
%%javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}

var connection = setInterval(ConnectButton, 60000);

function myStopFunction() {
    console.log("Remove interal"); 
    clearInterval(connection);
}

<IPython.core.display.Javascript object>

In [ ]:
import os
import json
import glob
import shutil

In [ ]:
%%capture
## install kaggle
!pip install kaggle

## make sure gdrive is mounted
if not os.path.exists('/content/drive'):
    from google.colab import drive
    drive.mount("/content/drive")

## pull repo from github
if not os.path.exists('/content/melanoma-classification'):
    !git clone https://github.com/SaschaMet/melanoma-classification.git

## update repo
!cd melanoma-classification && git fetch && git pull

# login to kaggle
with open('/content/drive/MyDrive/Colab Notebooks/_auth/kaggle.json') as json_file:
    data = json.load(json_file)
    os.environ['KAGGLE_USERNAME'] = data["username"]
    os.environ['KAGGLE_KEY'] = data["key"]

## download the dataset
if not os.path.exists('/content/melanoma-classification/data/test'):
    !cd /content/melanoma-classification/data && kaggle datasets download -d arroqc/siic-isic-224x224-images && unzip siic-isic-224x224-images.zip

!rm -rf /content/melanoma-classification/data/siic-isic-224x224-images.zip

In [ ]:
## switch branch and execute function
!cd /content/melanoma-classification && git checkout "add-train-function" && git fetch && git pull

M	train.py
Already on 'add-train-function'
Your branch is up to date with 'origin/add-train-function'.
Already up to date.


In [ ]:
!python /content/melanoma-classification/train.py

Tensorflow version 2.4.0
Running in colab: True
Mixed precision enabled
Accelerated Linear Algebra enabled
change BASE_PATH to  /content/melanoma-classification/data
Set seed
valid rows in train 33126
valid rows in test 10982
rows in train 33126
rows in test 10982
Samples in train 0.5
Remaining rows in train set 1168
create model
layer 1 : input_1 is trainable: False
layer 2 : block1_conv1 is trainable: False
layer 3 : block1_conv2 is trainable: False
layer 4 : block1_pool is trainable: False
layer 5 : block2_conv1 is trainable: False
layer 6 : block2_conv2 is trainable: False
layer 7 : block2_pool is trainable: False
layer 8 : block3_conv1 is trainable: False
layer 9 : block3_conv2 is trainable: False
layer 10 : block3_conv3 is trainable: False
layer 11 : block3_pool is trainable: False
layer 12 : block4_conv1 is trainable: False
layer 13 : block4_conv2 is trainable: False
layer 14 : block4_conv3 is trainable: True
layer 15 : block4_pool is trainable: True
layer 16 : block5_conv1 is t

In [ ]:
# remove the old results
!rm -rf /content/melanoma-classification/results

# create a new folder
!mkdir /content/melanoma-classification/results

# copy the training files to the results dir
os.chdir("/content")
for file in glob.glob("*.png"):
    fileName = file.split("-")[-1]
    shutil.move(file, "/content/melanoma-classification/evaluation/" + fileName)

for file in glob.glob("*.hdf5"):
    fileName = file.split("-")[-1]
    shutil.move(file, "/content/melanoma-classification/evaluation/" + fileName)

for file in glob.glob("*.json"):
    fileName = file.split("-")[-1]
    shutil.move(file, "/content/melanoma-classification/evaluation/" + fileName)

for file in glob.glob("*.txt"):
    shutil.move(file, "/content/melanoma-classification/evaluation/" + file)

In [ ]:
# push the files to github
## use gh auth token saved in gDrive
!cd melanoma-classification && git remote set-url origin $(head -n 1 "/content/drive/MyDrive/Colab Notebooks/_auth/gh_auth.txt")
!cd melanoma-classification && git status
!cd melanoma-classification && git add . && git commit -m "colab training" && git push